In [2]:
!pip install python-dotenv

# Getting data from Mergeflow's API

In [3]:
import requests
import json
import os
from dotenv import load_dotenv

load_dotenv()  # Load variables from .env file
mergeflow_api_key = os.getenv('MERGEFLOW_API_KEY')

# the API call
query = 'large+language+model' # topic is "large language model"
dataset = '&q=*%5bfundingcompany%7cfundingproject%7cfundingorganization%5d&sp=2572' # dataset: venture capital investments
rows = str(3) # first 3 docs only
url = 'https://mergeflow.net/api/v1/fw/search/getContent?q=' + query + dataset + '&rows=' + rows

# Define the headers to be sent with the request
headers = {
    'MergeflowNet-Api-Auth-Key': mergeflow_api_key
}

# Get and print the response
response = requests.post(url, headers=headers)

mergeflow_docs = []

# Check the response status code and content
if response.status_code == 200:
    mergeflow_response_json = response.json()
    
    # Pretty-print the JSON response -- useful to understand what the JSON looks like
    #pretty_json = json.dumps(mergeflow_response_json, indent=4)
    #print(pretty_json)
    
    # get title, URL, date, and content
    for document in mergeflow_response_json['Documents']:
        current_doc = {
            'title': document['Title'],
            'date': document['Date'],
            'url': document['Url'],
            'content': document['Content']
        }
        
        mergeflow_docs.append(current_doc)
        #print(content)        

    print('Found ' + str(len(mergeflow_docs)) + ' documents.')

else:
    print('Request failed with status code:', response.status_code)
    print(response.text)  # Print the response text


Found 3 documents.


# Answering questions on the results with OpenAI GPT

In [8]:
import openai

openai.api_key = os.getenv('OPENAI_API_KEY')

system_prompt = f"""You are Senior Vice President Technology and Innovation & CTO of Factory Automation. When you evaluate a new business idea, you strictly adhere to the following guiding principles:
        - Focus on Sense&Act.
        - Prioritize quick, low-investment idea testing.
        - Aim for large profits with smaller budgets, avoiding investments over 10-15 million per portfolio element.
        - Target a minimum 10 million EUR annual revenue.
        - Specialize in LLMs in Engineering and Operations.
        - Avoid new ecosystem creation.
        - Core business: Sense&Act, PMA, Scada, PLCs.
        - Prioritize product over solution business for scalability.
        - Consider AI as a potential product.
        - Ensure all initiatives align with business profitability.
        - Make investment decisions as if using personal funds.
        - Aim for non-cannibalizing, fringe markets.
        - PLCs are scalable; seek similar scalability in AI applications.
        - Validate ideas with business units and customer feedback.
        - Avoid focusing solely on infrastructure to ensure profitable outcomes.
        """

for doc in mergeflow_docs:
    user_prompt = f"""
    Consider the text below, delimited by ```, in the area of {query}:
    ```
    {doc['content']}
    ```
    What is the core idea of the text? Answer in 20 words or fewer.
    What applications does the text describe? Answer in 20 words or fewer.
    
    Use the following JSON format:
    {{
        "CoreIdea": "core idea of the text",
        "Appplications": "applications mentioned in the text"
    }}
    
    """
    
    completion = openai.chat.completions.create(
        model="gpt-3.5-turbo-0613",
        messages=[
            {"role": "system", "content": system_prompt},
            {"role": "user", "content": user_prompt}
        ],
        temperature=0.7,
        max_tokens=2000,
        top_p=0.95,
        frequency_penalty=0,
        presence_penalty=0,
        stop=None
    )

    print(completion.choices[0].message)

ChatCompletionMessage(content='{\n    "CoreIdea": "Peerlogic, an AI-driven dental practice technology company, raised $5.65M in funding to scale its impact and enhance its Large Language Model (LLM) for advanced call analytics and engagement tools.",\n    "Applications": "Large Language Model (LLM), advanced call analytics, engagement tools"\n}', role='assistant', function_call=None, tool_calls=None)
ChatCompletionMessage(content='{\n    "CoreIdea": "Robin AI, a provider of an AI-powered legal copilot, raised $26M in Series B funding to expand its team and extend its AI and machine learning capabilities.",\n    "Applications": "AI-powered legal copilot, automating contract drafting and negotiation, extracting information from contract repositories."\n}', role='assistant', function_call=None, tool_calls=None)
ChatCompletionMessage(content='{\n    "CoreIdea": "rabbit, an AI startup, has raised $10 million in a Series A round to develop a personalized operating system with a natural langu